In [1]:
import sys
import os
from datetime import datetime
import json
import shutil
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm_notebook, tqdm
from matplotlib import pyplot as plt

sys.path.append('../src/')
from env import BTCMarket_Env
from agent import Trader_Agent
from DQNtrainer import DQNTrainer
from DRLtrainer import DRLTrainer, DRLLossFunctions
from collections import deque


In [2]:
import agent as TAgent
import env as BTCEnv

from importlib import reload 

In [3]:
obs_space = (8,20)
act_space = 1

money = 20000
fee = 0.001
episodes = 1
runs_p_eps = 1
TAgent=reload(TAgent)
BTCEnv=reload(BTCEnv)
env = BTCEnv.BTCMarket_Env(observation_space = obs_space,
            action_space = act_space,
            start_money = money,
            trading_fee= fee)
agent = TAgent.Trader_Agent(observation_space = obs_space,
            action_space = act_space,
            epsilon = 0.01)

drltrainer = DRLTrainer(env, agent,
                observation_space = obs_space,
                action_space = act_space,
                batch_size=32,                        
                lstm_path="./best_models/11_mar_2023/best_model_sequential_20back_10ahead.h5")

# drltrainer.rollout(n_episodes=episodes, run_per_episode=runs_p_eps)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 8)]      0           []                               
                                                                                                  
 lstm_inputs (Lambda)           (None, 20, 4)        0           ['input_1[0][0]']                
                                                                                                  
 sequential (Sequential)        (None, 10)           1953940     ['lstm_inputs[0][0]']            
                                                                                                  
 flatten_1 (Flatten)            (None, 10)           0           ['sequential[0][0]']             
                                                                                              

In [4]:
env.reset()
data_samples = env.episode_length
memory = deque()
state_tmp = []
for t in range(20):
    state, _, _ = env.step(np.array([0]))
    state_tmp.append(state)
    tmp = agent.model(state)[0]
    print(tmp)

tf.Tensor([0.5600538], shape=(1,), dtype=float32)
tf.Tensor([0.5547038], shape=(1,), dtype=float32)
tf.Tensor([0.55212003], shape=(1,), dtype=float32)
tf.Tensor([0.5617867], shape=(1,), dtype=float32)
tf.Tensor([0.5708641], shape=(1,), dtype=float32)
tf.Tensor([0.54179853], shape=(1,), dtype=float32)
tf.Tensor([0.54871774], shape=(1,), dtype=float32)
tf.Tensor([0.5891282], shape=(1,), dtype=float32)
tf.Tensor([0.54052013], shape=(1,), dtype=float32)
tf.Tensor([0.5222809], shape=(1,), dtype=float32)
tf.Tensor([0.5430516], shape=(1,), dtype=float32)
tf.Tensor([0.5716828], shape=(1,), dtype=float32)
tf.Tensor([0.54545945], shape=(1,), dtype=float32)
tf.Tensor([0.5231729], shape=(1,), dtype=float32)
tf.Tensor([0.5705159], shape=(1,), dtype=float32)
tf.Tensor([0.5904391], shape=(1,), dtype=float32)
tf.Tensor([0.5966327], shape=(1,), dtype=float32)
tf.Tensor([0.5626761], shape=(1,), dtype=float32)
tf.Tensor([0.6032173], shape=(1,), dtype=float32)
tf.Tensor([0.5868097], shape=(1,), dtype=floa

In [5]:
state.shape

(1, 20, 8)

In [6]:
tmp = agent.model(state)[0]
tmp

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.5868097], dtype=float32)>

In [7]:
env.reset()
data_samples = env.episode_length
memory = deque()

state, _, _ = env.step(np.array([0]))
for t in tqdm(range(55)):
    # Compute Action
    tmp_wallet_value = env.wallet_value
    action = agent.compute_action(state)
    # Compute new step
    next_state, reward, done = env.step(action=action)
    # save Experience to Memory
    memory.append((state, action, reward, next_state, done))
    state = next_state

100%|██████████| 55/55 [00:06<00:00,  8.46it/s]


In [8]:
weights_0 = []
for layer in agent.model.layers:
    W = layer.get_weights()
    weights_0.append(W)

In [9]:
len(weights_0)

11

In [10]:
batch_size = 50


batch = []
# for i in range(1):
for i in range(max([len(memory) - batch_size,0]), len(memory)):
    batch.append(memory[i])

# Change here to fit new RL-Algo
# for index in range(0,len(batch)-1):

x_train = np.zeros((batch_size, 20,8))
y_train = np.zeros((batch_size, 4))
print(len(batch))


state, _, reward, next_state , done = batch[0]
action = agent.model.predict(state,verbose = 0)
for index in range(1,len(batch)):
    # Compute Reward Decay for DQN
    action_next = agent.model.predict(next_state,verbose = 0)
#     if not done:
#         reward += dqntrainer.gamma * np.max(action_next)
        
    if np.any(np.isnan(reward)) or np.any(np.isnan(action)):
        print(reward)
        print(action)
    target = np.nan_to_num(action,nan=0.0)      
    id_act = np.argmax(target)
    target[0,id_act] = reward
    
    target = np.nan_to_num(target,nan=0.0)
    state = np.nan_to_num(state,nan=0.0)
    x_train[index]= state[0]
    y_train[index]= target[0]
    
    
    # update state, action, reward for training
    state, _, reward, next_state , done = batch[index]
    action = action_next


result=agent.model.fit(x_train, y_train, epochs=5, verbose=1)
# agent.update_epsilon()


50
Epoch 1/5
2/2 [==============================] - 4s 7ms/step - loss: -0.0086
Epoch 2/5
2/2 [==============================] - 0s 5ms/step - loss: -0.0112
Epoch 3/5
2/2 [==============================] - 0s 5ms/step - loss: -0.0121
Epoch 4/5
2/2 [==============================] - 0s 4ms/step - loss: -0.0123
Epoch 5/5
2/2 [==============================] - 0s 4ms/step - loss: -0.0123


In [11]:
for t in range(20):
    tmp = agent.model(state_tmp[t])[0]
    print(tmp)

tf.Tensor([0.994418], shape=(1,), dtype=float32)
tf.Tensor([0.993891], shape=(1,), dtype=float32)
tf.Tensor([0.9937482], shape=(1,), dtype=float32)
tf.Tensor([0.9940692], shape=(1,), dtype=float32)
tf.Tensor([0.9928295], shape=(1,), dtype=float32)
tf.Tensor([0.9926745], shape=(1,), dtype=float32)
tf.Tensor([0.9927226], shape=(1,), dtype=float32)
tf.Tensor([0.9945404], shape=(1,), dtype=float32)
tf.Tensor([0.99320257], shape=(1,), dtype=float32)
tf.Tensor([0.9915154], shape=(1,), dtype=float32)
tf.Tensor([0.99405247], shape=(1,), dtype=float32)
tf.Tensor([0.9948067], shape=(1,), dtype=float32)
tf.Tensor([0.9952159], shape=(1,), dtype=float32)
tf.Tensor([0.9941586], shape=(1,), dtype=float32)
tf.Tensor([0.99629045], shape=(1,), dtype=float32)
tf.Tensor([0.9973701], shape=(1,), dtype=float32)
tf.Tensor([0.9970092], shape=(1,), dtype=float32)
tf.Tensor([0.9967571], shape=(1,), dtype=float32)
tf.Tensor([0.99769396], shape=(1,), dtype=float32)
tf.Tensor([0.99801147], shape=(1,), dtype=float3

In [12]:
weights_1 = []
for layer in agent.model.layers:
    W = layer.get_weights()
    weights_1.append(W)
print(len(weights_1))

11


In [13]:
keras.backend.clear_session()
for t in range(20):
    tmp = drltrainer.agent.model(state_tmp[t])[0]
    print(tmp)

tf.Tensor([0.994418], shape=(1,), dtype=float32)
tf.Tensor([0.993891], shape=(1,), dtype=float32)
tf.Tensor([0.9937482], shape=(1,), dtype=float32)
tf.Tensor([0.9940692], shape=(1,), dtype=float32)
tf.Tensor([0.9928295], shape=(1,), dtype=float32)
tf.Tensor([0.9926745], shape=(1,), dtype=float32)
tf.Tensor([0.9927226], shape=(1,), dtype=float32)
tf.Tensor([0.9945404], shape=(1,), dtype=float32)
tf.Tensor([0.99320257], shape=(1,), dtype=float32)
tf.Tensor([0.9915154], shape=(1,), dtype=float32)
tf.Tensor([0.99405247], shape=(1,), dtype=float32)
tf.Tensor([0.9948067], shape=(1,), dtype=float32)
tf.Tensor([0.9952159], shape=(1,), dtype=float32)
tf.Tensor([0.9941586], shape=(1,), dtype=float32)
tf.Tensor([0.99629045], shape=(1,), dtype=float32)
tf.Tensor([0.9973701], shape=(1,), dtype=float32)
tf.Tensor([0.9970092], shape=(1,), dtype=float32)
tf.Tensor([0.9967571], shape=(1,), dtype=float32)
tf.Tensor([0.99769396], shape=(1,), dtype=float32)
tf.Tensor([0.99801147], shape=(1,), dtype=float3

In [14]:
weights_2 = []
for layer in agent.model.layers:
    W = layer.get_weights()
    weights_2.append(W)
print(len(weights_2))

11


In [15]:
for i in range(len(weights_1)):
    print(len(weights_0[i]), len(weights_1[i]), len(weights_2[i]))

0 0 0
0 0 0
16 16 16
0 0 0
0 0 0
0 0 0
0 0 0
2 2 2
2 2 2
2 2 2
2 2 2


In [16]:
# agent.model.save("./tmp.h5")

In [17]:
n = 9
for i in range(len(weights_0[n])):
    print(np.linalg.norm(np.array(weights_0[n][i]) - np.array(weights_2[n][i])))

0.40332037
0.046550643


In [26]:
weights_2[9]

[array([[-0.06137011,  0.16692351,  0.12445207, ..., -0.17191455,
          0.03225005,  0.08871748],
        [ 0.13786386,  0.1666566 , -0.06891119, ..., -0.06563537,
          0.06799255,  0.01828112],
        [-0.02296421,  0.03993624,  0.15722962, ..., -0.0346944 ,
         -0.05199049,  0.01533619],
        ...,
        [-0.15001749,  0.09076492,  0.16815488, ..., -0.00492883,
          0.10139827, -0.13099168],
        [-0.17542817,  0.05263472, -0.12932709, ..., -0.083545  ,
          0.03416753,  0.10203524],
        [-0.0962092 ,  0.02072592, -0.07874995, ...,  0.00170738,
         -0.11721523, -0.14244227]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [27]:
model1 = keras.models.load_model("./../data/20230402_194342/DRL_trial_0models_ai_trade_20230402_210314_5.h5", compile=False)
model1.compile(loss=DRLLossFunctions)
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 8)]      0           []                               
                                                                                                  
 lstm_inputs (Lambda)           (None, 20, 4)        0           ['input_1[0][0]']                
                                                                                                  
 sequential (Sequential)        (None, 10)           1953940     ['lstm_inputs[0][0]']            
                                                                                                  
 flatten_1 (Flatten)            (None, 10)           0           ['sequential[0][0]']             
                                                                                              

In [28]:
weights_2 = []
for layer in model1.layers:
    W = layer.get_weights()
    weights_2.append(W)
print(len(weights_2))

11


In [29]:
model2 = keras.models.load_model("./../data/20230402_194342/DRL_trial_0models_ai_trade_20230403_113557_50.h5", compile=False)
model2.compile(loss=DRLLossFunctions)
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 8)]      0           []                               
                                                                                                  
 lstm_inputs (Lambda)           (None, 20, 4)        0           ['input_1[0][0]']                
                                                                                                  
 sequential (Sequential)        (None, 10)           1953940     ['lstm_inputs[0][0]']            
                                                                                                  
 flatten_1 (Flatten)            (None, 10)           0           ['sequential[0][0]']             
                                                                                              

In [30]:
weights_3 = []
for layer in model2.layers:
    W = layer.get_weights()
    weights_3.append(W)
print(len(weights_3))

11


In [31]:
n = 9
for i in range(len(weights_3[n])):
    print(np.linalg.norm(np.array(weights_3[n][i]) - np.array(weights_2[n][i])))

0.0
0.0


In [32]:
weights_3[n]

[array([[-0.03737549,  0.02916929,  0.07700019, ..., -0.01748151,
         -0.15335801,  0.14982112],
        [ 0.01506725, -0.05898755,  0.04430667, ...,  0.12692027,
         -0.02553394,  0.1561246 ],
        [-0.10220589, -0.08384782, -0.16171944, ...,  0.03401756,
          0.13757946, -0.15559061],
        ...,
        [-0.08259476,  0.13240947, -0.15867913, ...,  0.09194656,
          0.04406597, -0.16755913],
        [ 0.15152867, -0.05129798, -0.07526167, ...,  0.16436686,
          0.15782975, -0.08786227],
        [-0.10800084, -0.07619185, -0.07103641, ...,  0.12390186,
          0.10318042, -0.04645446]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [33]:
weights_2[n]

[array([[-0.03737549,  0.02916929,  0.07700019, ..., -0.01748151,
         -0.15335801,  0.14982112],
        [ 0.01506725, -0.05898755,  0.04430667, ...,  0.12692027,
         -0.02553394,  0.1561246 ],
        [-0.10220589, -0.08384782, -0.16171944, ...,  0.03401756,
          0.13757946, -0.15559061],
        ...,
        [-0.08259476,  0.13240947, -0.15867913, ...,  0.09194656,
          0.04406597, -0.16755913],
        [ 0.15152867, -0.05129798, -0.07526167, ...,  0.16436686,
          0.15782975, -0.08786227],
        [-0.10800084, -0.07619185, -0.07103641, ...,  0.12390186,
          0.10318042, -0.04645446]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]